# Leer registros

Los objetivos de aprendizaje son:

1. Comando `SELECT`
    - SQL
    - SQLModel
        - Función `select()`
        - Método `.exec()`
        - Clase `Results`
        - Método `.all()`
2. COMANDO `WHERE`
    - SQL
    - SQLModel
        - Clase `Select`
        - Método `.where()`
        - Operadores Lógicos


## Comando `SELECT`

### SQL

Antes de escribir código en Python, hagamos una revisión rápida de cómo leer registros en SQL:

```SQL
SELECT id, nombre, lob, age FROM asegurado;

```

Podemos usar `*` para seleccionar todos los campos:

```SQL
SELECT * FROM asegurado;
```

Podemos sólo seleccionar un subconjunto de campos:


```SQL
SELECT id, nombre FROM asegurado;
```

Podemos indicar explicitamente de qué tabla tomar cada campo:

```SQL
SELECT a.id FROM asegurado as a;
```

### SQLModel

El primer paso es crear un `engine`, de la misma manera que hicimos al crear las filas.

In [1]:
from sqlmodel import Session, create_engine

sqlite_file_name = "database.db"
sqlite_url = f"sqlite:///{sqlite_file_name}"

engine = create_engine(sqlite_url, echo=True)

#### Función `select()`

Ahora crearemos una cláusula `SELECT` pero con SQLModel.

Primero tenemos que importar la función `select`:

In [2]:
from sqlmodel import select

Luego usaremos la función `select` dentro de la cáusula `with` de python que nos ayudará a gestionar la sesión con la base de datos:

``` python
with Session(engine) as session:
        statement = select(Asegurado)
```

Pasamos el modelo de datos `Asegurado` a la función `select()`, que es equivalente a seleccionar todas las columnas y registros de la tabla `asegurado`.


#### Método `.exec()`

Ahora que tenemos la instrucción select dentro de la variable `statement`, podemos usar la sesión para enviar a la APIDB el comando y que ésta se encarge de ejecutarlo y regresarnos los resultados:

```python
with Session(engine) as session:
        statement = select(Asegurado)
        results = session.exec(statement)

```

#### Clase `Results`

La variable `results` es una instancia de la clase `Results`, que es un iterable que nos permite ir uno a uno por todos los registros que ha generado nuestra consulta a la base de datos.

In [3]:
from crear_base_de_datos import Asegurado

with Session(engine) as session:
        statement = select(Asegurado)
        results = session.exec(statement)
        asegurados = [a for a in results]

2024-02-05 19:15:08,595 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-05 19:15:08,597 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("asegurado")
2024-02-05 19:15:08,597 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-02-05 19:15:08,598 INFO sqlalchemy.engine.Engine COMMIT
2024-02-05 19:15:08,600 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-05 19:15:08,605 INFO sqlalchemy.engine.Engine SELECT asegurado.id, asegurado.nombre, asegurado.lob, asegurado.edad 
FROM asegurado
2024-02-05 19:15:08,606 INFO sqlalchemy.engine.Engine [generated in 0.00113s] ()
2024-02-05 19:15:08,607 INFO sqlalchemy.engine.Engine ROLLBACK


In [4]:
asegurados

[Asegurado(lob='Autos', edad=30, id=1, nombre='Heber'),
 Asegurado(lob='Hogar', edad=35, id=2, nombre='Joaquim'),
 Asegurado(lob='Autos', edad=40, id=3, nombre='Antonio'),
 Asegurado(lob='Hogar', edad=25, id=4, nombre='Juan')]

In [5]:
from crear_base_de_datos import Asegurado

with Session(engine) as session:
        statement = select(Asegurado)
        asegurados = session.exec(statement).all()

2024-02-05 19:15:21,843 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-05 19:15:21,845 INFO sqlalchemy.engine.Engine SELECT asegurado.id, asegurado.nombre, asegurado.lob, asegurado.edad 
FROM asegurado
2024-02-05 19:15:21,846 INFO sqlalchemy.engine.Engine [cached since 13.24s ago] ()
2024-02-05 19:15:21,848 INFO sqlalchemy.engine.Engine ROLLBACK


In [6]:
asegurados

[Asegurado(lob='Autos', edad=30, id=1, nombre='Heber'),
 Asegurado(lob='Hogar', edad=35, id=2, nombre='Joaquim'),
 Asegurado(lob='Autos', edad=40, id=3, nombre='Antonio'),
 Asegurado(lob='Hogar', edad=25, id=4, nombre='Juan')]

Podemos crear un `DataFrame` usando los resultados:

In [7]:
import pandas as pd
pd.DataFrame.from_records([i.dict() for i in asegurados])

,lob,edad,id,nombre
0,Autos,30,1,Heber
1,Hogar,35,2,Joaquim
2,Autos,40,3,Antonio
3,Hogar,25,4,Juan


## COMANDO `WHERE`

### SQL 

SQL admite la cláusula `WHERE` para filtrar datos:

``` SQL
SELECT * FROM asegurado WHERE id=1;

SELECT * FROM asegurado WHERE id=1 AND nombre='Heber' OR lob='Hogar';
```


### SQLModel

#### Clase `Select`

la función `select()`regresa una instancia de la clase `SelectOfScalar`, esta clase contiene distintos métodos.

#### Método `.where()`

De la misma manera que agregamos `WHERE` a una instrucción SQL para filtrar filas, podemos agregar el método `.where()` a la instancia de la clase `SelectOfScalar` para filtrar filas:


In [8]:
from crear_base_de_datos import Asegurado

with Session(engine) as session:
        statement = select(Asegurado).where(Asegurado.lob == "Autos")
        asegurados = session.exec(statement).all()
asegurados

2024-02-05 19:16:39,500 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-05 19:16:39,501 INFO sqlalchemy.engine.Engine SELECT asegurado.id, asegurado.nombre, asegurado.lob, asegurado.edad 
FROM asegurado 
WHERE asegurado.lob = ?
2024-02-05 19:16:39,502 INFO sqlalchemy.engine.Engine [generated in 0.00054s] ('Autos',)
2024-02-05 19:16:39,504 INFO sqlalchemy.engine.Engine ROLLBACK


[Asegurado(lob='Autos', edad=30, id=1, nombre='Heber'),
 Asegurado(lob='Autos', edad=40, id=3, nombre='Antonio')]

El método `.where()` regresa una instancia de la clase `SelectOfScalar`, así que podemos concatener múltipls llamadas:

In [10]:
from crear_base_de_datos import Asegurado

with Session(engine) as session:
        statement = select(Asegurado).where(Asegurado.lob == "Autos").where(Asegurado.edad <35)
        asegurados = session.exec(statement).all()
asegurados

2023-02-06 19:27:37,668 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-02-06 19:27:37,671 INFO sqlalchemy.engine.Engine SELECT asegurado.id, asegurado.nombre, asegurado.lob, asegurado.edad 
FROM asegurado 
WHERE asegurado.lob = ? AND asegurado.edad < ?
2023-02-06 19:27:37,672 INFO sqlalchemy.engine.Engine [generated in 0.00139s] ('Autos', 35)
2023-02-06 19:27:37,674 INFO sqlalchemy.engine.Engine ROLLBACK


[Asegurado(edad=30, id=1, nombre='Heber', lob='Autos')]

#### Operadores Lógicos

Hemos visto que la expresión:

```python
select(Asegurado).where(Asegurado.lob == "Autos").where(Asegurado.edad <35)
````

Es equivalente a:

```SQL
SELECT * FROM asegurado WHERE asegurado.lob = 'AUTOS' AND asegurado.edad < 35;
```

¿Qué pasa si queremos usar `OR`?¿Qué pasa si queremos ser más explícitos y usar `AND`?

Podemos usar los operdores lógicos `or_` y `and_`

In [11]:
from sqlmodel import and_, or_

with Session(engine) as session:
        statement = select(Asegurado).where(
            and_(
                Asegurado.lob == "Autos",
                Asegurado.edad <35,
            )
        )
        asegurados = session.exec(statement).all()
asegurados

2023-02-06 19:29:05,480 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-02-06 19:29:05,483 INFO sqlalchemy.engine.Engine SELECT asegurado.id, asegurado.nombre, asegurado.lob, asegurado.edad 
FROM asegurado 
WHERE asegurado.lob = ? AND asegurado.edad < ?
2023-02-06 19:29:05,484 INFO sqlalchemy.engine.Engine [generated in 0.00132s] ('Autos', 35)
2023-02-06 19:29:05,486 INFO sqlalchemy.engine.Engine ROLLBACK


[Asegurado(edad=30, id=1, nombre='Heber', lob='Autos')]

In [52]:
from sqlmodel import and_, or_

with Session(engine) as session:
        statement = select(Asegurado).where(
            or_(
                Asegurado.lob == "Autos",
                Asegurado.nombre == "Juan",
            )
        )
        asegurados = session.exec(statement).all()
asegurados

2023-01-28 13:11:48,681 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-01-28 13:11:48,683 INFO sqlalchemy.engine.Engine SELECT asegurado.id, asegurado.nombre, asegurado.lob, asegurado.edad 
FROM asegurado 
WHERE asegurado.lob = ? OR asegurado.nombre = ?
2023-01-28 13:11:48,685 INFO sqlalchemy.engine.Engine [cached since 189.3s ago] ('Autos', 'Juan')
2023-01-28 13:11:48,688 INFO sqlalchemy.engine.Engine ROLLBACK


[Asegurado(id=1, edad=30, nombre='Heber', lob='Autos'),
 Asegurado(id=3, edad=40, nombre='Antonio', lob='Autos'),
 Asegurado(id=4, edad=25, nombre='Juan', lob='Hogar')]